In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [3]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [4]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'shape_area_new'}, inplace=True)  # old:new.
ca_bg_joined.rename(columns={'CNTY_NAME':'cnty_name'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25607


In [5]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")

# renaming 
ca_flares.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

print(f'Flares found: {len(ca_flares)}')

Flares found: 117


In [6]:
ca_flares.geometry

0      POINT (101702.097 -392465.466)
1       POINT (50045.686 -404732.075)
2      POINT (111804.156 -403954.218)
3       POINT (67527.269 -411223.881)
4       POINT (62783.972 -410781.392)
                    ...              
112    POINT (165197.510 -469510.426)
113    POINT (165197.511 -469609.028)
114     POINT (-239947.045 35040.908)
115    POINT (245587.514 -437618.082)
116    POINT (225403.428 -439194.020)
Name: geometry, Length: 117, dtype: geometry

In [7]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [8]:
# set col list for BCM_avg calculation
bcm_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[bcm_list].mean(axis=1) 

In [9]:
# subset to only columns needed for Tableau
col_list = ['flare_id','flare_cate','cnty_name','BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021','BCM_avg', 'geometry']

subset = ca_flares[col_list].copy()

# assign the subsetted copy back to the original variable name to keep total # of df names small 
# throughout project
ca_flares = subset

In [10]:
ca_flares.sample(1)

,flare_id,flare_cate,cnty_name,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2017,BCM_2018,BCM_2019,BCM_2020,BCM_2021,BCM_avg,geometry
45,36ecd8cd107104,flares_upstream,Kern County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,POINT (94072.137 -332054.081)


### Update ca_bg_joined df

In [11]:
# There are five BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000, 60379903000, 60599901000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

In [12]:
# subset to only cols needed
subset = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'shape_area_new', 'geometry']].copy()

# assign the subsetted copy back to the original variable name
ca_bg_joined = subset

ca_bg_joined.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

In [13]:
# counties_sub = ca_counties[['cnty_name', 'geometry']].copy()

# counties_sub.rename(columns={'geometry':'cnty_geom'}, inplace=True)  # old:new. Match col names for merging

# # add county names to social dataframe
# ca_bg_joined_sub = pd.merge(ca_bg_joined_sub, counties_sub, 
#                              on='cnty_name', how='left')

#### DFs to work with for aggregating by BG and aggregating by flare buffer

In [14]:
ca_bg_joined.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom
1237,60371438012,Los Angeles County,1219,442,22.56,"403,840.68","POLYGON ((148335.636 -429180.203, 148365.222 -..."


In [15]:
ca_flares.sample(1)

,flare_id,flare_cate,cnty_name,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2017,BCM_2018,BCM_2019,BCM_2020,BCM_2021,BCM_avg,geometry
23,324f5181547428,flares_upstream,Ventura County,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,POINT (60196.612 -410134.074)


# Aggregate data by BG and export shapefile for Tableau

In [16]:
# take baseline dfs and create copies clearly defined for this BG analysis, separate from the work that is needed
# for the flare buffer analysis below. Both analyses modify these same two dfs in different ways and I want 
# to make sure I keep the calculations separated

bg_socialdf = ca_bg_joined.copy()
bg_flaresdf = ca_flares.copy()

# set geometry columns
bg_socialdf = bg_socialdf.set_geometry('bg_geom')

# and rename for clarity
bg_flaresdf.rename(columns={'geometry':'flarepts_geom'}, inplace=True)  # old:new. Match col names for merging
bg_flaresdf = bg_flaresdf.set_geometry('flarepts_geom')

In [17]:
bg_flaresdf.columns

Index(['flare_id', 'flare_cate', 'cnty_name', 'BCM_2012', 'BCM_2013',
       'BCM_2014', 'BCM_2015', 'BCM_2016', 'BCM_2017', 'BCM_2018', 'BCM_2019',
       'BCM_2020', 'BCM_2021', 'BCM_avg', 'flarepts_geom'],
      dtype='object')

In [18]:
#flares_df = set_geometry_buffer(flares_df, buffer_size)
buffer_size=2000
buffer_col = f"buff{buffer_size}_geom"
bg_flaresdf[buffer_col] = bg_flaresdf['flarepts_geom'].buffer(distance=buffer_size)

bg_flaresdf = bg_flaresdf.set_geometry('buff2000_geom')


In [19]:
# subset of desired columns from flares_df
flares_sjoin_subset = bg_flaresdf[['buff2000_geom', 'BCM_avg', 'flare_id', 'flare_cate']]

# Left join the flares_sjoin_subset cols to the bg_socialdf based on if any of the flares geometries intersect 
# with the bg geometries
bg_intersect = gp.sjoin(bg_socialdf, flares_sjoin_subset, how='left', predicate='intersects')


In [20]:
# Bring in buffer col that is not brought in by the sjoin() function for some unknown reason. Frustrating.
bg_intersect = pd.merge(bg_intersect, flares_sjoin_subset[['flare_id', 'buff2000_geom']], 
                             on='flare_id', how='left')

In [21]:
len(bg_intersect)

25865

In [22]:
# Iterate through each bg_id and flare_id pairing and calculate 
# the proportion of the buffer that intersects with the BG geom, relative to the total buffer size.
# We need to know the proportion of each buffer in each bg geom in order to assign the percentage of
# flare volume to each bg in the case of incomplete overlap.

# First check for null or invalid geometries in buffer_2000m column
bg_intersect['buffer_valid'] = bg_intersect['buff2000_geom'].is_valid

# Initialize intersect proportion "b_int_prop" and intersect area "b_int_area" columns with NaN values
bg_intersect['buff_int_prop'] = np.nan
bg_intersect['buff_int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in bg_intersect.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row['buff2000_geom'].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row['buff2000_geom'].area
        # Calculate intersection proportion
        buffer_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        bg_intersect.at[index, 'buff_int_prop'] = buffer_int_prop
        bg_intersect.at[index, 'buff_int_area'] = intersection_area
        bg_intersect.at[index, 'buff_area'] = buffer_area

    else:
        # Assign 0 to b_int_prop and b_int_area if buffer_2000m geometry is invalid
        bg_intersect.at[index, 'buff_int_prop'] = 0
        bg_intersect.at[index, 'buff_int_area'] = 0
        bg_intersect.at[index, 'buff_area'] = 0

        
# Drop buffer_valid column as it's no longer needed
bg_intersect.drop('buffer_valid', axis=1, inplace=True)



### I confirm this works as intended in the Folium mapping at the bottom of the notebook

In [23]:
len(bg_intersect)

25865

In [24]:
# Now a similar analysis but this time we want to know the BG-buffer intersection proportion relative to 
# the total area of the block group, in order to use that proportion to assign counts of BG demographic
# variables to the intersection area. 

# Combine all flares into a single multipolygon unit
temp = bg_flaresdf.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=bg_flaresdf.crs)  # convert back to geodf for processing

# Make sure our geom column is set correctly to "bg_geom" rather than the flare buffer
if bg_intersect.geometry.name == "bg_geom":
    int_temp = gp.overlay(bg_intersect, all_buffers, how='intersection')  
    int_temp.rename(columns={'geometry':'bg_int_geom'}, inplace=True)  # old:new. For clarity
    # Set geometry to the intersections
    int_temp = int_temp.set_geometry('bg_int_geom')  

    # Create new 'area' column for the areas of the intersections
    int_temp['bg_int_area'] = int_temp.area

    # Calculate the proportion of each block group intersection relative to total block broup area
    int_temp['bg_int_prop'] = int_temp['bg_int_area'] / int_temp['shape_area_new']

    # Merge the new area and props variables
    bg_intersect = pd.merge(bg_intersect, 
                                 int_temp[['bg_id','bg_int_prop','bg_int_geom']], 
                                 on=['bg_id'], 
                                 how='left').fillna({'bg_int_prop': 0})
    
else:
    print("Error: Geometry column is not named 'bg_geom'")

In [25]:
bg_intersect.sample(50)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom,index_right,BCM_avg,flare_id,flare_cate,buff2000_geom,buff_int_prop,buff_int_area,buff_area,bg_int_prop,bg_int_geom
8,60730101101,San Diego County,1833,1608,47.67,"531,684.08","POLYGON ((273377.487 -599666.365, 273379.173 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
9558,60710099101,San Bernardino County,2508,1765,31.78,"1,305,260.97","POLYGON ((243144.683 -390507.772, 243144.588 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
11057,60670079033,Sacramento County,660,39,11.05,"289,285.27","POLYGON ((-114365.618 72368.819, -114294.426 7...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
921,60372699081,Los Angeles County,1184,720,34.66,"118,098.18","POLYGON ((146186.344 -441429.611, 146215.334 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
18815,60730199043,San Diego County,2154,1001,37.21,"459,637.28","POLYGON ((257118.950 -535030.086, 257132.578 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
18464,60730083572,San Diego County,1115,813,45.26,"197,618.75","POLYGON ((268199.319 -561991.523, 268214.944 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
4907,60470022012,Merced County,625,557,65.12,"659,923.90","POLYGON ((-75064.582 -105775.495, -75036.131 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
27195,61110096002,Ventura County,2906,707,9.92,"24,853,672.44","POLYGON ((67738.131 -414975.718, 67833.685 -41...",30.00,0.00,6295afae420679,flares_upstream,"POLYGON ((69346.315 -411230.717, 69336.684 -41...",0.06,"692,699.48","12,546,193.96",0.04,"POLYGON ((68253.538 -413085.630, 68251.998 -41..."
26339,60650427493,Riverside County,2852,2021,40.65,"2,217,363.61","POLYGON ((248064.851 -475460.168, 248120.818 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None
19549,60014301013,Alameda County,1655,952,27.93,"21,873,701.24","POLYGON ((-179572.922 -31480.187, -179556.487 ...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None


In [27]:
# Apply the proportion to each demographic variable to find counts by variable
# i.e. this answers "how many of x live in the intersection area?"
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    bg_intersect[var + '_int_count'] = bg_intersect[var] * bg_intersect['bg_int_prop']

# Find overall proportions for each demo var by dividing var count by respective total population
# i.e. this answers "what percentage of this block group is minority status?"
bg_intersect['MINORPOP_bg_totprop'] = bg_intersect['MINORPOP'] / bg_intersect['ACSTOTPOP']

# calculate percentage of minorities in intersection for each BG
bg_intersect['bg_int_mnr_prop'] = bg_intersect['MINORPOP_int_count'] / bg_intersect['ACSTOTPOP_int_count']

In [28]:
# check calculations against two examples of non-intersected BGs and two of intersected BGs
bg_intersect[bg_intersect['flare_id'].isna()].head(2).append(bg_intersect[~bg_intersect['flare_id'].isna()].head(2))

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom,index_right,BCM_avg,flare_id,flare_cate,buff2000_geom,buff_int_prop,buff_int_area,buff_area,bg_int_prop,bg_int_geom,ACSTOTPOP_int_count,MINORPOP_int_count,MINORPOP_bg_totprop,bg_int_mnr_prop
0,60376500012,Los Angeles County,864,576,34.33,"279,785.40","POLYGON ((154828.423 -457797.384, 154927.488 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None,0.00,0.00,0.67,NaN
1,60376500014,Los Angeles County,748,646,46.59,"297,744.27","POLYGON ((154836.126 -458134.127, 154835.934 -...",NaN,NaN,NaN,NaN,None,0.00,0.00,0.00,0.00,None,0.00,0.00,0.86,NaN
4,60376503005,Los Angeles County,1102,838,44.91,"733,298.96","POLYGON ((151858.050 -459765.962, 151869.518 -...",110.00,0.00,36c62e67595483,flares_oil_downstream,"POLYGON ((156983.129 -460981.588, 156973.498 -...",0.01,"83,271.03","12,546,193.96",0.11,"POLYGON ((153266.503 -459960.018, 153266.880 -...",125.14,95.16,0.76,0.76
333,60375780004,Los Angeles County,1095,938,84.26,"1,074,800.40","POLYGON ((164176.032 -467882.392, 164192.453 -...",113.00,0.00,857477a0194312,flares_oil_downstream,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.02,"237,504.03","12,546,193.96",0.53,"MULTIPOLYGON (((164176.032 -467882.392, 164192...",582.59,499.06,0.86,0.86


#### Standardize variables

EJ index and population are already encoded at the level of a BG, so they're ready to normalize. 

Flare volume needs to multiplied by the proportion of the flare's buffer zone that resides in a BG, and then all (in the case of multiple flares in a BG) adjusted flare volumes summed up to generate a variable for total flare volume a BG is exposed to.  

In [ ]:
def calc_adj_flare_vol(df):
    """
    "bcm_adj" = adjusted bcm value for a given flare_id and bg_id, based on the proportion of the buffer zone that 
    overlaps with a given bg_id. Equal to the bcm_avg * b_int_prop (the overlap)
    
    "bcm_adj_tot" = the sum of the bcm_adj values for a given bg_id
    """
    
    # calculate the product of bcm_avg and b_int_prop for each flare and bg combination
    df['bcm_adj'] = df['BCM_avg'] * df['b_int_prop']
    # sum the adjusted volumes for all flare and bg combinations
    adj_vol = df.groupby(['flare_id', 'bg_id'])['bcm_adj'].sum().reset_index()
    # sum the adjusted volumes for all flares in the block group
    bg_vol = adj_vol.groupby('bg_id')['bcm_adj'].sum().reset_index()
    # merge the block group volumes with the original dataframe
    df = pd.merge(df, bg_vol, on='bg_id', how='left', suffixes=('', '_tot'))
    return df


In [ ]:
intersect_all = intersect_all.groupby('bg_id').apply(calc_adj_flare_vol)

In [ ]:
# Assuming your dataframe is named `intersect_all`
df_uniq = intersect_all.drop_duplicates(subset=['bg_id', 'flare_id'])

In [ ]:
type(df_uniq)

In [ ]:
df_uniq.columns

In [ ]:
# Define the variables to standardize

#'bcm_adj_tot' = Sum of the adjusted flare volume(s) that intersect w the BG. 
#'D_PM25_2' = EJ index for the BG
#'ACSTOTPOP_int_cnt' = BG population that resides in the intersection. = Totpop * intersection_proportion

varlist = ['bcm_adj_tot', 'D_PM25_2', 'ACSTOTPOP_int_cnt']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(df_uniq[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_norm' for var in varlist])

intersect_norm = intersect_norm.reset_index(drop=True)
df_uniq = df_uniq.reset_index(drop=True)

# Concatenate the new variables with the original DataFrame
df_uniq = pd.concat([df_uniq, intersect_norm], axis=1)


In [ ]:
df_uniq.columns

In [ ]:
# check calculations against two BGs
# Should see a single "bcm_adj_tot" value per BG, and different adjusted "bcm_adj" values per flare_id 
#depending on the amount of overlap.

# filter the dataframe and format specified columns to display 4 decimal places
filtered_df = df_uniq[df_uniq['bg_id'].isin([60379800331, 60379800111])][['bg_id', 'flare_id', 'BCM_avg', 'b_int_prop', 'bcm_adj', 'bcm_adj_tot','bcm_adj_tot_norm','bg_int_prop','ACSTOTPOP','MINORPOP','ACSTOTPOP_int_cnt','MINORPOP_int_cnt', 'int_mnr_pc','b_int_area', 'buff_area']]
filtered_df[['BCM_avg', 'bcm_adj', 'bcm_adj_tot','bcm_adj_tot_norm']] = filtered_df[['BCM_avg', 'bcm_adj', 'bcm_adj_tot','bcm_adj_tot_norm']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))

In [ ]:
filtered_df

In [ ]:
df_uniq[df_uniq['flare_id'].isna()].head(2).append(df_uniq[~df_uniq['flare_id'].isna()].head(2))


In [ ]:
# save shapefile for visualization in Tableau

df_uniq.rename(columns={'shape_area_new':'shape_area'}, inplace=True)  # old:new.
df_uniq.rename(columns={'intersect_prop':'int_prop'}, inplace=True)  # old:new.
df_uniq.rename(columns={'ACSTOTPOP_int_cnt':'pop'}, inplace=True)  # old:new.
df_uniq.rename(columns={'ACSTOTPOP_int_cnt_norm':'pop_norm'}, inplace=True)  # old:new.
#df_uniq.rename(columns={'MINORPOP_int_cnt':'int_mnrpop'}, inplace=True)  # old:new.
df_uniq.rename(columns={'bcm_adj_tot_norm':'bcm_norm'}, inplace=True)  # old:new.
df_uniq.rename(columns={'D_PM25_2':'pm25'}, inplace=True)  # old:new.
df_uniq.rename(columns={'D_PM25_2_norm':'pm25_norm'}, inplace=True)  # old:new.
df_uniq.rename(columns={'bg_int_geom':'int_geom'}, inplace=True)  # old:new.


# rename the flare categories for readability
df_uniq["flare_cate"] = df_uniq["flare_cate"].replace({"flares_oil_downstream": "down_oil",
                                                        "flares_upstream": "upstream"})

In [ ]:
df_uniq = df_uniq.set_geometry('int_geom')

In [ ]:
df_uniq.drop('bg_geom', axis=1, inplace=True)
df_uniq.drop('buff2000_geom', axis=1, inplace=True)

In [ ]:
df_uniq.columns

In [ ]:
# num BGs
len(df_uniq.loc[df_uniq['flare_id'].notnull(), 'bg_id'].dropna().unique())

In [ ]:
# num flares
len(df_uniq['flare_id'].dropna().unique())

In [ ]:
df_uniq['int_geom'].geom_type.unique()

In [ ]:
df_uniq.to_file("data/df_bg_impactmetric_shp.shp", driver='ESRI Shapefile')

# Aggregate by Flare ID and export shapefile for Tableau

### Create buffer

In [ ]:
def create_buffer_intersection(flares_df, social_df, buffer_size):
    """
    Creates a new Geodf containing the intersection between census BGs
    and buffers of {buffer size} around flare sites.
    
    Parameters
    ----------
    flares_df : A Geodf containing point geometries representing flare sites.
    social_df : A Geodf containing polygon geometries representing social data from EPA EJScreen
    buffer_size : int
        The size of the buffer around each flare site, in meters.
    
    Returns
    -------
    tuple of GeoDataFrames
        A tuple containing two GeoDataFrames:
        1. The original flares_df with a new buffer column added.
        2. A GeoDataFrame containing the intersection between social_df and the 
        union of all buffer geometries in flares_df.
    """
    
    social_df = social_df.set_geometry('bg_geom')
    
    #flares_df = set_geometry_buffer(flares_df, buffer_size)
    buffer_col = f"buffer_{buffer_size}m"
    flares_df[buffer_col] = flares_df['geometry'].buffer(distance=buffer_size)

    flares_df = flares_df.set_geometry(buffer_col)
    
    temp = flares_df.unary_union
    all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=flares_df.crs)  # convert back to geodf for processing
    
    intersect_temp = gp.overlay(social_df, all_buffers, how='intersection')  # could look at keep_geom=False
    
    intersect_temp.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging
    
    intersect_temp = intersect_temp.set_geometry('intersect_geom')

    intersect_temp2 = gp.sjoin(intersect_temp, flares_df, how = "left", predicate = 'intersects')

    return flares_df, intersect_temp2

In [ ]:
buffer_sizes = [2000]
#buffer_sizes = [100, 400, 800, 1000, 1600, 2000, 5000, 7500, 10000]

for buffer_size in buffer_sizes:
    ca_flares_new, intersect = create_buffer_intersection(ca_flares_sub, ca_bg_joined_sub, buffer_size)

In [ ]:
intersect.columns

In [ ]:
intersect.sample(1)

In [ ]:
# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_all[var + '_int_cnt'] = intersect_all[var] * intersect_all['bg_int_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    intersect_all[var + '_bg_totprop'] = intersect_all[var] / intersect_all['ACSTOTPOP']

In [ ]:
# grab only necessary cols
intersect_forflares = intersect_final[['flare_id','bg_id', 'cnty_name_left', 'flare_cate',
                                 'BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015','BCM_2016', 
                                 'BCM_2017', 'BCM_2018', 'BCM_2019', 'BCM_2020', 'BCM_2021','BCM_avg',
                                 'buffer_2000m','pop','pm25','bcm_norm']].copy()

In [ ]:
# Define the variables to aggregate
agg_dict = {'pm25': 'mean', 'pop': 'sum'}

# Group the block groups by flare ID and calculate the aggregation for the variables
ca_flares_agg = intersect_forflares.groupby('flare_id').agg(agg_dict)

# Rename the output variables
ca_flares_agg = ca_flares_agg.rename(columns={'pm25': 'pm25_avg', 'pop': 'pop_sum'})

# Join the aggregated variables with the original columns
ca_flares_merged = intersect_forflares.merge(ca_flares_agg, on='flare_id')


In [ ]:
# Define the variables to standardize
varlist = ['pm25_avg', 'pop_sum']  # because bcm is already attached to the flare unit of analysis

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(ca_flares_merged[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
ca_flares_merged_f = pd.concat([ca_flares_merged, intersect_norm], axis=1)

In [ ]:
ca_flares_merged_f.sample(3)

In [ ]:
ca_flares_merged_f = gp.GeoDataFrame(ca_flares_merged_f, geometry='buffer_2000m', crs=meters_crs)


In [ ]:
ca_flares_merged_f.to_file("data/df_flare_impactmetric_shp.shp", driver='ESRI Shapefile')

## Calculating single instance of weights here in python

In [ ]:
# # Define the weights for each variable
# bcm_weight = 0
# pm25_weight = 1
# pop_weight = 0

In [ ]:
# # Calculate the weighted variables
# intersect['BCM_weighted'] = intersect['BCM_avg_norm'] * bcm_weight
# intersect['D_PM25_2_weighted'] = intersect['D_PM25_2_norm'] * pm25_weight
# intersect['ACSTOTPOP_weighted'] = intersect['ACSTOTPOP_intersect_count_norm'] * pop_weight

# # Define the variables to sum
# varlist_weighted = ['BCM_weighted', 'D_PM25_2_weighted', 'ACSTOTPOP_weighted']

# # Group the block groups by flare ID and sum the weighted variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_weighted].sum()

# # Calculate the impact metric as the sum of the weighted variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_weighted].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# # Show the results
# ca_flares_merged.head()

In [ ]:
# # Define the variables to sum
# varlist_norm = ['BCM_avg_norm', 'D_PM25_2_norm', 'ACSTOTPOP_intersect_count_norm']

# # Group the block groups by flare ID and sum the standardized variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_norm].sum()

# # Calculate the impact metric as the sum of the standardized variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_norm].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# df_final = pd.merge(ca_flares_merged, intersect, on='flare_id')

In [ ]:
# invalid_geoms = df_final[~df_final.is_valid]

In [ ]:
# invalid_geoms

In [ ]:
# # save for use in tableau
# df_final.to_csv(F"data/df_impactmetric_csv.csv", index=False)

In [ ]:
# # check for missing values
# missing_values = df_final.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

In [ ]:
# df_final.sample(2)

## Top ten

In [ ]:
# # display the top ten flares by impact metric
# top_ten = for_map.nlargest(10, 'impact_metric')
# top_ten = top_ten.set_geometry('buffer_2000m')

In [ ]:
# # check for missing values
# missing_values = for_map.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

## Folium Mapping

In [ ]:
# for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
# for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

# for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

In [ ]:

# Define the color scale and number of bins
#color_scale = 'Reds'
#num_bins = 10

# Create a map centered on the first flare
# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)




In [ ]:
# Feature Group: Buffers

def style_function3(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'fillOpacity': 0.05
    }

inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=test["buffer_2000m"],
    style_function=style_function3
).add_to(inter_all_bg)
inter_all_bg.add_to(m)

In [ ]:
# Feature Group: block group

def style_function2(feature):
    return {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.05
    }

bg_bound = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=test["bg_geom"],
    style_function=style_function2
).add_to(bg_bound)
bg_bound.add_to(m)

In [ ]:
# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = for_map,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
#              'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)

In [ ]:
# Display the map
m